# Preparando a base de dados

Essa é a primeira parte do trabalho de analisar as eleições estaduais de 2018. Nesse notebook, vamos pegar a base de dados original do TSE, selecionar os candidatos que atingiram uma votação igual a pelo menos 10% da quota eleitoral e corrigir o problema de formatação dos acentos ortográficos.

O próximo passo, que é limpar a base, rodar o modelo estatístico e analisar os resultados estarão em outro notebook.

In [1]:
#importing libraries

import numpy as np
import pandas as pd

In [2]:
#importando a base
df = pd.read_csv(".\\Bases de dado intermediárias\\sp2018.df.csv")

In [3]:
# vamos selecionar os candidatos que atingiram uma votação igual a pelo menos 10% da quota eleitoral:
df = df[df['qe.10'] == 1]

In [4]:
# veja como aparece códigos como <c9> e <c7> nos nomes dos candidatos
df.head()

,Unnamed: 0,CD_MUNICIPIO,NM_MUNICIPIO,NM_CANDIDATO,SG_PARTIDO,NR_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,DS_SIT_TOT_TURNO,votoscandmun,...,DESCRICAO_OCUPACAO,DESPESA_MAX_CAMPANHA,COD_SIT_TOT_TURNO,DESC_SIT_TOT_TURNO,SITUACAO_REELEICAO,SITUACAO_DECLARAR_BENS,NUMERO_PROTOCOLO_CANDIDATURA,NUMERO_PROCESSO,TITULO_CONFERIDO,qe.10
3,4,61000,EUCLIDES DA CUNHA PAULISTA,AD<c9>RMIS MARINI JUNIOR,PSDB,4533,APTO,DEFERIDO,SUPLENTE,1,...,ECONOMISTA,0.0,5.0,SUPLENTE,N,S,-1.0,6.020899e+18,1.693398e+11,1
17,18,61000,EUCLIDES DA CUNHA PAULISTA,ADILSON BARROSO OLIVEIRA,PATRI,5151,APTO,DEFERIDO,N<c3>O ELEITO,0,...,EMPRES<c1>RIO,0.0,4.0,N<c3>O ELEITO,N,S,-1.0,6.016646e+18,4.814854e+10,1
23,24,61000,EUCLIDES DA CUNHA PAULISTA,ADRIANA MIGUEL VENTURA,NOVO,3050,APTO,DEFERIDO,ELEITO POR QP,0,...,ADMINISTRADOR,0.0,2.0,ELEITO POR QP,N,S,-1.0,6.015546e+18,1.672880e+11,1
29,30,61000,EUCLIDES DA CUNHA PAULISTA,ADRIANO ELI CORR<ca>A,DEM,2577,APTO,DEFERIDO,ELEITO POR M<c9>DIA,62,...,DEPUTADO,0.0,3.0,ELEITO POR M<c9>DIA,S,S,-1.0,6.020907e+18,2.542277e+11,1
39,40,61000,EUCLIDES DA CUNHA PAULISTA,AILTON JOS<c9> DE LIMA,PSD,5545,APTO,DEFERIDO,SUPLENTE,0,...,EMPRES<c1>RIO,0.0,5.0,SUPLENTE,N,S,-1.0,6.020916e+18,1.276201e+11,1


#### Eu não encontrei em nenhum lugar da internet um correspondente entre os códigos que aparecem na base de dados e os acentos ortográficos. Assim sendo, eu tive que ir na raça, descobrindo os acentos, colocando em um dicionário, substituindo todo mundo e vendo se sobrou algum.

In [5]:
# vendo a base de dados, eu consegui inicialmente deduzir os seguintes acentos:
acentos = {'<cd>' : 'Í',
           "<c9>" : "É",
           "<c3>" : "Ã",
           "<c1>" : "Á",
           "<e7>" : "Ç",
           "<f5>" : "Õ",
           "<ca>" : "Ê",
           "<c7>" : "Ç",
           '<d3>' : 'Ó'}

#### Para substituir os valores dos acentos, temos que fazer um loop com todas as colunas. Como eu não encontrei uma forma de fazer um loop direto nas colunas, eu fiz um loop para printar os códigos que eu precisava e, em seguida, usei esse codigo:

In [6]:
# loop que, para cada coluna, substitui os acentos do dicionário "acentos"
for i in df.columns:
    for key, value in acentos.items():
        df.loc[:,i] = df.loc[:,i].str.replace(key, value)

AttributeError: Can only use .str accessor with string values!

#### Deu errado porque o .str só funciona com colunas que sejam string. Assim sendo, vamos procurar as colunas que são de texto:

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87075 entries, 3 to 1022939
Data columns (total 53 columns):
Unnamed: 0                        87075 non-null int64
CD_MUNICIPIO                      87075 non-null int64
NM_MUNICIPIO                      87075 non-null object
NM_CANDIDATO                      87075 non-null object
SG_PARTIDO                        87075 non-null object
NR_CANDIDATO                      87075 non-null int64
DS_SITUACAO_CANDIDATURA           87075 non-null object
DS_DETALHE_SITUACAO_CAND          87075 non-null object
DS_SIT_TOT_TURNO                  87075 non-null object
votoscandmun                      87075 non-null int64
votoscanduf                       87075 non-null int64
ANO_ELEICAO                       87075 non-null float64
COD_TIPO_ELEICAO                  87075 non-null float64
DESCRICAO_ELEICAO                 87075 non-null object
SIGLA_UF                          87075 non-null object
DESCRICAO_UE                      87075 non-null obj

#### Veja que do total de 53 colunas, 30 são colunas de texto. Vamos usar o comando abaixo para fazermos uma lista de colunas de texto:

In [8]:
# fazendo uma lista das colunas que são objeto
colunas_texto = df.select_dtypes(include='object').columns

In [9]:
# loop que, para cada coluna, substitui os acentos do dicionário "acentos"
for i in colunas_texto:
    for key, value in acentos.items():
        df.loc[:,i] = df.loc[:,i].str.replace(key, value)

In [10]:
# vamos ver o resultado
df[colunas_texto].head()

,NM_MUNICIPIO,NM_CANDIDATO,SG_PARTIDO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,DS_SIT_TOT_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,DESCRICAO_UE,DESCRICAO_CARGO,...,NOME_MUNICIPIO_NASCIMENTO,DATA_NASCIMENTO,DESCRICAO_SEXO,DESCRICAO_GRAU_INSTRUCAO,DESCRICAO_ESTADO_CIVIL,DESCRICAO_COR_RACA,DESCRICAO_OCUPACAO,DESC_SIT_TOT_TURNO,SITUACAO_REELEICAO,SITUACAO_DECLARAR_BENS
3,EUCLIDES DA CUNHA PAULISTA,ADÉRMIS MARINI JUNIOR,PSDB,APTO,DEFERIDO,SUPLENTE,EleiÇÕes Gerais Estaduais 2018,SP,SÃO PAULO,DEPUTADO FEDERAL,...,FRANCA,18/07/1969,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,ECONOMISTA,SUPLENTE,N,S
17,EUCLIDES DA CUNHA PAULISTA,ADILSON BARROSO OLIVEIRA,PATRI,APTO,DEFERIDO,NÃO ELEITO,EleiÇÕes Gerais Estaduais 2018,SP,SÃO PAULO,DEPUTADO FEDERAL,...,MINAS NOVAS,14/06/1964,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA,EMPRESÁRIO,NÃO ELEITO,N,S
23,EUCLIDES DA CUNHA PAULISTA,ADRIANA MIGUEL VENTURA,NOVO,APTO,DEFERIDO,ELEITO POR QP,EleiÇÕes Gerais Estaduais 2018,SP,SÃO PAULO,DEPUTADO FEDERAL,...,SÃO PAULO,06/03/1969,FEMININO,SUPERIOR COMPLETO,CASADO(A),BRANCA,ADMINISTRADOR,ELEITO POR QP,N,S
29,EUCLIDES DA CUNHA PAULISTA,ADRIANO ELI CORRÊA,DEM,APTO,DEFERIDO,ELEITO POR MÉDIA,EleiÇÕes Gerais Estaduais 2018,SP,SÃO PAULO,DEPUTADO FEDERAL,...,SÃO PAULO,13/01/1976,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,DEPUTADO,ELEITO POR MÉDIA,S,S
39,EUCLIDES DA CUNHA PAULISTA,AILTON JOSÉ DE LIMA,PSD,APTO,DEFERIDO,SUPLENTE,EleiÇÕes Gerais Estaduais 2018,SP,SÃO PAULO,DEPUTADO FEDERAL,...,INAJÁ,27/10/1965,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),PARDA,EMPRESÁRIO,SUPLENTE,N,S


#### Agora vamos verificar se sobrou alguma linha com código ao invés de acento

In [11]:
# vamos fazer uma lista dos nomes

nomes = []

for coluna in colunas_texto:
    nomes.append(df.loc[:,coluna].unique())

# planificando a lista
nomes = [x for l in nomes for x in l]

In [12]:
# como os códigos vem sempre em formato <código>, vamos procurar por "<"
codigos = []
lista = []

for nome in nomes: 
    if "<" in nome:
        codigos.append(nome.split("<")[1].split('>')[0])
        lista.append(nome)
        
pd.Series(codigos).unique()

array(['da', 'c2', 'd4', 'd5', 'e3'], dtype=object)

#### Sabemos agora que faltam os códigos 'da', 'c2', 'd4', 'd5', 'e3'

In [13]:
# vamos descobrir a correspondência dos acentos que sobraram

lista = []
for nome in nomes: 
    if "<" in nome:
        lista.append(nome)
lista

['EMBA<da>BA',
 'ALVINL<c2>NDIA',
 'SANTO ANT<d4>NIO DO ARACANGUÁ',
 'ANAL<c2>NDIA',
 'HORTOL<c2>NDIA',
 'PRAT<c2>NIA',
 'PAULIST<c2>NIA',
 'BOM JESUS DOS PERD<d5>ES',
 'BRA<da>NA',
 'CABRE<da>VA',
 'CAFEL<c2>NDIA',
 'C<c2>NDIDO MOTA',
 'C<c2>NDIDO RODRIGUES',
 'COL<d4>MBIA',
 'DIVINOL<c2>NDIA',
 'GUZOL<c2>NDIA',
 'HERCUL<c2>NDIA',
 'IBI<da>NA',
 'IN<da>BIA PAULISTA',
 'IPE<da>NA',
 'JAGUARI<da>NA',
 'JA<da>',
 'J<da>LIO MESQUITA',
 'LUÍS ANT<d4>NIO',
 'LUIZI<c2>NIA',
 'MACED<d4>NIA',
 'MIRASSOL<c2>NDIA',
 'MONÇ<d5>ES',
 'NOVA LUZIT<c2>NIA',
 'ORINDI<da>VA',
 'ORL<c2>NDIA',
 'RIOL<c2>NDIA',
 'SANTA L<da>CIA',
 'SANTO ANT<d4>NIO DA ALEGRIA',
 'SANTO ANT<d4>NIO DE POSSE',
 'SANTO ANT<d4>NIO DO JARDIM',
 'SANTO ANT<d4>NIO DO PINHAL',
 'TAI<da>VA',
 'TAMBA<da>',
 'TURI<da>BA',
 'UR<c2>NIA',
 'GUILHERME CAMPOS J<da>NIOR',
 'SAMUEL MOREIRA DA SILVA J<da>NIOR',
 'GUILHERME CAMPOS J<da>NIOR',
 'SAMUEL MOREIRA DA SILVA J<da>NIOR',
 'S<c2>MIA BOMFIM',
 'PARTIDO DA REP<da>BLICA',
 'ColigaÇ<e3>o P

In [14]:
# adicionando a correspondência no dicionário
acentos['<d5>'] = 'Õ'
acentos['<e3>'] = 'Ã'
acentos['<da>'] = 'Ú'
acentos['<c2>'] = 'Â'
acentos['<d4>'] = 'Ô'


# rodando tudo de novo:


# loop que, para cada coluna, substitui os acentos do dicionário "acentos"
for i in colunas_texto:
    for key, value in acentos.items():
        df.loc[:,i] = df.loc[:,i].str.replace(key, value)
    
    
# vamos fazer uma lista dos nomes
nomes = []
for coluna in colunas_texto:
    nomes.append(df.loc[:,coluna].unique())

# planificando a lista
nomes = [x for l in nomes for x in l]


# como os códigos vem sempre em formato <código>, vamos procurar por "<"
codigos = []
lista = []
for nome in nomes: 
    if "<" in nome:
        codigos.append(nome.split("<")[1].split('>')[0])
        lista.append(nome)
        
pd.Series(codigos).unique()

array(['e1'], dtype=object)

#### Quase pronto! Só faltou o e1

In [15]:
# adicionando a correspondência no dicionário
acentos['<e1>'] = 'Á'



# rodando tudo de novo:


# loop que, para cada coluna, substitui os acentos do dicionário "acentos"
for i in colunas_texto:
    for key, value in acentos.items():
        df.loc[:,i] = df.loc[:,i].str.replace(key, value)
    
    
# vamos fazer uma lista dos nomes
nomes = []
for coluna in colunas_texto:
    nomes.append(df.loc[:,coluna].unique())

# planificando a lista
nomes = [x for l in nomes for x in l]


# como os códigos vem sempre em formato <código>, vamos procurar por "<"
codigos = []
lista = []
for nome in nomes: 
    if "<" in nome:
        codigos.append(nome.split("<")[1].split('>')[0])
        lista.append(nome)
        
pd.Series(codigos).unique()

array([], dtype=float64)

### Perfeito! Vamos guardar o resultado como "Tudo Acentuado Finalmente"

In [16]:
df.to_csv(".\\Bases de dado intermediárias\\Tudo Acentuado Finalmente.csv")